In [31]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

def save_to_csv(file_name):
	data = pd.io.stata.read_stata(file_name + '.dta')
	data.to_csv(file_name + '.csv')

def extract_label_and_feature_column(file_name):
	df = pd.read_csv(file_name)
	return df['WHO_WAS_CONTACTED'], df['COMMENTS_CLOB']

def numerize_label(string):
	if string != string:
		return 0
	if 'Client' in string:
		return 1
	if 'Client' not in string and 'Collateral' in string:
		return 3
	if 'Client' not in string:
		return 2
		
def process_label_and_feature(df,X):
	df = df.apply(lambda x: numerize_label(x))
	na_values = get_na_values(X)
	df = df.drop(na_values)
	X = X.drop(na_values)
	return df,X

def get_na_values(X):
	return np.where(X.isna())[0].tolist()

def clear(x):
    x=str(x)
    x=x.lower()
    word_tokens=nltk.word_tokenize(x)
    word_tokens=[lemmatizer.lemmatize(x) for x in word_tokens]
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    return filtered_sentence 



In [2]:
Y, X = extract_label_and_feature_column('DYS_Casenotes_16Oct20_merged_final_bydischarge.csv')
Y, X = process_label_and_feature(Y,X)
stop_words = set(stopwords.words('english'))

/Users/lw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,44,46) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
X = X.apply(clear)
from collections import Counter
all_token=[]
for x in X:
    all_token+=x
Counter(all_token).most_common(100)

[('.', 430031),
 ('wa', 108291),
 (',', 99941),
 ('cw', 55223),
 ("'s", 46599),
 ('home', 46287),
 ('ha', 36586),
 ('school', 36398),
 ('mother', 33010),
 ('met', 32018),
 ('today', 30115),
 ('reported', 29784),
 ('spoke', 29714),
 ('would', 29447),
 ('program', 28153),
 ('client', 24462),
 ('well', 24311),
 ('meeting', 24072),
 ('caseworker', 23770),
 ('also', 23062),
 ('time', 22013),
 ('youth', 21665),
 ('called', 20815),
 ('stated', 20112),
 ('informed', 19116),
 ('work', 18860),
 ('report', 18748),
 ('going', 18548),
 ('week', 16915),
 ('back', 16906),
 ('get', 16842),
 ('call', 16114),
 ('office', 16093),
 ('writer', 16070),
 ('issue', 15817),
 ('go', 15740),
 ('went', 15544),
 ('asked', 15518),
 ('visit', 15162),
 (')', 14895),
 ('discussed', 14881),
 ('(', 14750),
 ('court', 14383),
 ('told', 14361),
 ('mom', 14255),
 ('day', 14178),
 ('house', 14130),
 ('said', 14038),
 ('know', 13702),
 ('dy', 12683),
 ('next', 12638),
 ('check', 12555),
 ('could', 11840),
 ('-', 11394),
 ('j

In [4]:
addlist=['.',',','cw','spoke','!','?']
stop_words = set(stopwords.words('english'))
for x in addlist:
    stop_words.add(x)

In [5]:
data = X.values.tolist()


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['spoke', 'clerk', 'office', 'regarding', 'danny', 'going', 'physically', 'transported', 'court', 'today', 'sheriff_department', 'case', 'pm']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en',disable=['parser', 'ner'])


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['regard', 'danny', 'go', 'physically', 'transport', 'court', 'today', 'case', 'pm']]


In [10]:
print(len(data_lemmatized))

140148


In [11]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.054*"program" + 0.031*"meeting" + 0.029*"client" + 0.025*"discuss" + '
  '0.023*"case" + 0.022*"family" + 0.019*"next" + 0.017*"sign" + '
  '0.017*"transport" + 0.016*"court"'),
 (1,
  '0.082*"go" + 0.061*"report" + 0.058*"meet" + 0.053*"home" + 0.045*"today" + '
  '0.041*"work" + 0.033*"well" + 0.030*"visit" + 0.026*"week" + 0.026*"youth"'),
 (2,
  '0.027*"would" + 0.027*"call" + 0.021*"also" + 0.021*"school" + '
  '0.021*"state" + 0.021*"mother" + 0.017*"ask" + 0.016*"inform" + '
  '0.016*"tell" + 0.015*"say"'),
 (3,
  '0.049*"weekend" + 0.047*"text" + 0.043*"charge" + 0.035*"incident" + '
  '0.032*"approve" + 0.028*"involve" + 0.026*"curfew" + 0.018*"communicate" + '
  '0.016*"card" + 0.015*"event"')]


In [14]:
from gensim.test.utils import datapath
lda_model.save('lda.model')



In [15]:
import pickle
with open("test.txt", "wb") as fp:   #Pickling
    pickle.dump(corpus, fp)
with open("test.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print(len(b))

140148


In [16]:
predicted_labels = lda_model.get_document_topics(corpus)

In [17]:
print(predicted_labels[0])

[(0, 0.33527306), (1, 0.25884923), (2, 0.37620497), (3, 0.029672703)]


In [18]:
new_label = []
prob = []
for i in predicted_labels:
    label,probability = max(i, key=lambda x:x[1])
    new_label.append(label)
    prob.append(probability)

In [19]:
new_label = np.array(new_label)

In [20]:
new_label_0 = np.where(new_label == 0)[0]
new_label_1 = np.where(new_label == 1)[0]
new_label_2 = np.where(new_label == 2)[0]
new_label_3 = np.where(new_label == 3)[0]
Y_0 = np.where(Y == 0)[0]
Y_1 = np.where(Y == 1)[0]
Y_2 = np.where(Y == 2)[0]
Y_3 = np.where(Y == 3)[0]

In [21]:
print(max(Y))
print(Y_0.shape,Y_1.shape,Y_2.shape,Y_3.shape)
print("staffing, youth, no_youth, collateral")
print(new_label_0.shape,new_label_1.shape,new_label_2.shape,new_label_3.shape)

3
(3063,) (85109,) (21217,) (30759,)
staffing, youth, no_youth, collateral
(11199,) (15013,) (113915,) (21,)


In [22]:
s = set(Y_1)
s1 =set (new_label_2)
print(len(s.intersection(s1)))  
print(113914-85109)
#portion of Y_1 belongs to new_label_2

64538
28805


In [23]:
s = set(Y_2)
s1 = set(new_label_1)
print(len(s.intersection(s1)))  

1088


In [24]:
s = set(Y_2)
s1 = set(new_label_2)
print(len(s.intersection(s1)))  
# Most Y_2 belongs to new_label_2

19371


In [25]:
s = set(Y_3)
s1 = set(new_label_2)
print(len(s.intersection(s1)))
# Y_3 belongs to new_label_2

27745


In [26]:
s = set(Y_1)
s1 = set(new_label_1)
print(len(s.intersection(s1)))  
#The other portion of Y_1 belongs to new_label_1

12620


In [27]:
s = set(Y_2)
s1 = set(new_label_2)
print(len(s.intersection(s1)))  
# Most of Y_0 belongs to new_label_2

19371


In [28]:
s = set(Y_0)
s1 = set(new_label_2)
print(len(s.intersection(s1)))  

2261


In [29]:
s = set(Y_1)
s1 = set(new_label_0)
print(len(s.intersection(s1)))  

7934


In [30]:
# Conclusion is that collateral, staffing notes, and only parent/guardian present
# should be combined, there is no difference. 
# the Youth and also youth with other people present should be divided into 3 categories. 
# further analysis needed.